## Data cleaning and feature engineering

### 1. Preprocessing raw data

In [11]:
import glob, os

import pandas as pd
import numpy as np

In [12]:
from IPython.display import display
pd.options.display.max_columns = None

In [13]:
#Concatenate 7 years into 1 evaluation and 1 employee table
eva = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "eval_201*.csv"))))
emp = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "employee_201*.csv"))))

In [14]:
#Merge the eva and emp tables into 1 Dataframe (df), on ['employee_id', 'year'] key combination
df = pd.merge(eva, emp, how='outer', on=['employee_id', 'year'])

In [15]:
#Converting the age_range column from string to numeric values. For example '30-39' age range will become 3. 
#People less than 20 years old are assigned 1 and people older than 70 years old are assigned 7.
df['age_range'] = df['age_range'].astype(str).str[0]

df['age_range'] = df['age_range'].str.replace('>','7')
df['age_range'] = df['age_range'].str.replace('<','1')
df['age_range'] = df['age_range'].str.replace('n','')

df['age_range'] = pd.to_numeric(df['age_range'])

In [16]:
#Some department (fn_area) changed name, thus some cleaning needs to be made.
df.replace({'LEGAL & REGULATIONS':'LEGAL & COMPLIANCE', 'RESEARCH & DEVELOPMENT':'RESEARCH & INNOVATION', 'SUPPLY CHAIN & PURCHASING':'SUPPLY CHAIN', 'PUBLIC AFFAIRS & COMMUNICATION':'PUBLIC AFFAIRS'}, inplace=True)

In [17]:
#Assign a label to each function area
x = sorted(df['fn_area'].dropna().unique())

d = {}
for i in range(19):
    d[x[i]] = 'area'+str(i+1)
d

{'COMMERCIAL': 'area1',
 'COMMUNICATION': 'area2',
 'FINANCE': 'area3',
 'GENERAL MANAGEMENT': 'area4',
 'HSEQ AND ORGANISATIONAL PERFORMANCE': 'area5',
 'HUMAN RESOURCES': 'area6',
 'INDUSTRIAL': 'area7',
 'INFORMATION & KNOWLEDGE': 'area8',
 'INTELLECTUAL ASSETS': 'area9',
 'LEGAL & COMPLIANCE': 'area10',
 'MANUFACTURING': 'area11',
 'MARKETING AND SALES': 'area12',
 'Not assigned': 'area13',
 'PROCUREMENT': 'area14',
 'PUBLIC AFFAIRS': 'area15',
 'QUALITY & PERFORMANCE': 'area16',
 'RESEARCH & INNOVATION': 'area17',
 'SUPPLY CHAIN': 'area18',
 'SUPPORT': 'area19'}

In [18]:
#Sort the df by employee_id and then year, which is the most efficient structure for manipulating data
df.sort_values(by=['employee_id', 'year'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.tail()

### 2. Imputing missing values

In [19]:
#It is much more efficient to create shifting columns and then do vectorized operation, then looping through the whole df.
df['prev_employee_id'] = df['employee_id'].shift()
df['next_employee_id'] = df['employee_id'].shift(periods=-1)

#### Filling age_range

In [20]:
#if an employee has at least 1 record with age and other records missing age, use the available age to fill the missing records.
for i in range(8):
    df['prev_age'] = df['age_range'].shift()
    df['next_age'] = df['age_range'].shift(periods=-1)

    df['age_range'] = np.where(df['age_range'].isnull() & (df['employee_id'] == df['prev_employee_id']), df['prev_age'], df['age_range'])
    df['age_range'] = np.where(df['age_range'].isnull() & (df['employee_id'] == df['next_employee_id']), df['next_age'], df['age_range'])

#if an employee has no record with age, first determine if the employee is or used to be a manager
list_managers = df['manager_id'].unique()
df['is_manager'] = np.where(df['employee_id'].isin(list_managers), 1, 0)

#next calculate the average age of manager-employees and nonmanager-employees
unique_emp_first = df[~df.duplicated(subset=['employee_id'], keep='first')]
unique_emp_last = df[~df.duplicated(subset=['employee_id'], keep='last')]

avg_age_manager = (unique_emp_first[unique_emp_first['is_manager'] == 1]['age_range'].mean() + unique_emp_last[unique_emp_last['is_manager'] == 1]['age_range'].mean())/2
avg_age_non_manager = (unique_emp_first[unique_emp_first['is_manager'] == 0]['age_range'].mean() + unique_emp_last[unique_emp_last['is_manager'] == 0]['age_range'].mean())/2

#use the average ages to fill the missing records accordingly.
df['age_range'] = np.where(df['age_range'].isnull() & (df['is_manager'] == 1), avg_age_manager, np.where(df['age_range'].isnull() & (df['is_manager'] == 0), avg_age_non_manager, df['age_range']))

#### Filling fn_class and seniority_fn

In [21]:
#This is the main function that fills in missing values in fn_class and seniority_fn. 
    #The loop is to ensure new correct information is updated after each iteration. There are 8 years so we run it 8 times.
def fn_loop():
    for i in range(8):
        df['prev_fn_class'] = df['fn_class'].shift()
        df['next_fn_class'] = df['fn_class'].shift(periods=-1)

        df['prev_seniority_fn'] = df['seniority_fn'].shift()
        df['next_seniority_fn'] = df['seniority_fn'].shift(periods=-1)
        
        #Many records have missing fn_class. However, some can be implied with 99% confidence 
            #by looking at the previous year and/or next year record.
        df['fn_class'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['fn_class'].isnull()) & (df['seniority_fn'].notnull()), df['prev_fn_class'], df['fn_class'])
        df['fn_class'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['fn_class'].isnull()) & (df['next_seniority_fn'].notnull()) & (df['next_seniority_fn'] > 11), df['next_fn_class'], df['fn_class'])
        df['fn_class'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['employee_id'] == df['prev_employee_id']) & (df['fn_class'].isnull()) & (df['next_seniority_fn'].notnull()) & (df['next_seniority_fn'] <= 11), df['prev_fn_class'], df['fn_class'])
        df['fn_class'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['fn_class'].isnull()) & (df['seniority_fn'].notnull()), df['next_fn_class'], df['fn_class'])

        for j in range(8):
            df['prev_seniority_fn'] = df['seniority_fn'].shift()
            df['next_seniority_fn'] = df['seniority_fn'].shift(periods=-1)

            df['prev_year'] = df['year'].shift()
            df['next_year'] = df['year'].shift(periods=-1)
            
            #Many missing records in seniority_fn also follow certain patterns and can be be filled with these logics.
            df['seniority_fn'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['seniority_fn'].isnull()) & (df['next_seniority_fn'] == 12), 0, df['seniority_fn'])
            df['seniority_fn'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['seniority_fn'].isnull()) & (df['fn_class'].notnull()) & (df['next_seniority_fn']-(df['next_year'] - df['year'])*12 >= 0), (df['next_seniority_fn']-(df['next_year'] - df['year'])*12), df['seniority_fn'])
            df['seniority_fn'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['employee_id'] == df['prev_employee_id']) & (df['seniority_fn'].isnull()) & (df['fn_class'].notnull()) & (df['next_seniority_fn']-(df['next_year'] - df['year'])*12 < 0), (df['prev_seniority_fn']+(df['year'] - df['prev_year'])*12), df['seniority_fn'])
            df['seniority_fn'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['seniority_fn'].notnull()) & (df['fn_class'] == df['prev_fn_class']) & (df['seniority_fn'] < (df['prev_seniority_fn']+12)), (df['prev_seniority_fn']+(df['year'] - df['prev_year'])*12), df['seniority_fn'])

fn_loop()

In [22]:
#If the above logics still can't solve missing seniority and the employee has a function class,
    #we assume that it is his first year in the current class and assign 11 to it.
def seniority_fn():
    df['seniority_fn'] = np.where(df['fn_class'].notnull() & df['seniority_fn'].isnull(), 11, df['seniority_fn'])

seniority_fn()
#run the fn_loop again to ensure there is no inconsistency after the seniority_fn update
fn_loop()

In [23]:
#Here we check the fn_class of the employee's manager
man_emp = df[df['is_manager'] == 1]

mapper = man_emp[['employee_id', 'year', 'fn_class']].set_index(['employee_id', 'year'])['fn_class'].to_dict()
df['manager_fn_class'] = df.set_index(['manager_id', 'year']).index.map(mapper.get)

#If our confident logics still can't solve missing fn_class and the employee has a seniority_fn number,
    #we assume that he must have a fn_class and assign the manager_fn_class minus 3 to him.
    #Our rationale is that manager_fn_class usually equals employee's fn_class + 3.
df['fn_class'] = np.where(df['fn_class'].isnull() & df['seniority_fn'].notnull() & df['manager_fn_class'].notnull(), df['manager_fn_class']-3, df['fn_class'])

#However fn_class cannot get below 15. Hence the check.
df['fn_class'] = np.where(df['fn_class']<15, 15, df['fn_class'])

#### Filling bus_supp, fn_area, job_id and some leftover fn_class

In [24]:
#Here we find the employee's manager's bus_supp and fn_area
mapper = man_emp[['employee_id', 'year', 'bus_supp']].set_index(['employee_id', 'year'])['bus_supp'].to_dict()
df['manager_bus_supp'] = df.set_index(['manager_id', 'year']).index.map(mapper.get)

mapper = man_emp[['employee_id', 'year', 'fn_area']].set_index(['employee_id', 'year'])['fn_area'].to_dict()
df['manager_fn_area'] = df.set_index(['manager_id', 'year']).index.map(mapper.get)

df['prev_manager_id'] = df['manager_id'].shift()
df['next_manager_id'] = df['manager_id'].shift(periods=-1)

#We'll need to use this shift function many times
def update2():    
    df['prev_fn_class'] = df['fn_class'].shift()
    df['next_fn_class'] = df['fn_class'].shift(periods=-1)

    df['prev_bus_supp'] = df['bus_supp'].shift()
    df['next_bus_supp'] = df['bus_supp'].shift(periods=-1)

    df['prev_fn_area'] = df['fn_area'].shift()
    df['next_fn_area'] = df['fn_area'].shift(periods=-1)

    df['prev_job_id'] = df['job_id'].shift()
    df['next_job_id'] = df['job_id'].shift(periods=-1)

In [25]:
#Here we fill the bus_supp, fn_area, job_id and some more fn_class. 
    #Our best guess is that those features will be the same if the employee still have the same manager.
for i in range(8):
    update2()

    df['fn_class'] = np.where((df['fn_class'].isnull()) & (df['manager_id'] == df['prev_manager_id']) & (df['employee_id'] == df['prev_employee_id']), df['prev_fn_class'], df['fn_class'])
    df['fn_class'] = np.where((df['fn_class'].isnull()) & (df['manager_id'] == df['next_manager_id']) & (df['employee_id'] == df['next_employee_id']), df['next_fn_class'], df['fn_class'])

    df['bus_supp'] = np.where((df['bus_supp'].isnull()) & (df['manager_id'] == df['prev_manager_id']) & (df['employee_id'] == df['prev_employee_id']), df['prev_bus_supp'], df['bus_supp'])
    df['bus_supp'] = np.where((df['bus_supp'].isnull()) & (df['manager_id'] == df['next_manager_id']) & (df['employee_id'] == df['next_employee_id']), df['next_bus_supp'], df['bus_supp'])

    df['fn_area'] = np.where((df['fn_area'].isnull()) & (df['manager_id'] == df['prev_manager_id']) & (df['employee_id'] == df['prev_employee_id']), df['prev_fn_area'], df['fn_area'])
    df['fn_area'] = np.where((df['fn_area'].isnull()) & (df['manager_id'] == df['next_manager_id']) & (df['employee_id'] == df['next_employee_id']), df['next_fn_area'], df['fn_area'])

    df['job_id'] = np.where((df['job_id'].isnull()) & (df['manager_id'] == df['prev_manager_id']) & (df['employee_id'] == df['prev_employee_id']), df['prev_job_id'], df['job_id'])
    df['job_id'] = np.where((df['job_id'].isnull()) & (df['manager_id'] == df['next_manager_id']) & (df['employee_id'] == df['next_employee_id']), df['next_job_id'], df['job_id'])

In [26]:
#Here we try to fill the same things, but more agressive this time. We want to target those observations with an obj 
    #or noc score, since those observations are valuable and we want to retain as much information as possible.
for j in range(8):
    update2()

    df['fn_class'] = np.where((df['fn_class'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['prev_fn_class'], df['fn_class'])
    df['fn_class'] = np.where((df['fn_class'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['next_employee_id']), df['next_fn_class'], df['fn_class'])

    df['bus_supp'] = np.where((df['bus_supp'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['prev_bus_supp'], df['bus_supp'])
    df['bus_supp'] = np.where((df['bus_supp'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['next_bus_supp'], df['bus_supp'])

    df['fn_area'] = np.where((df['fn_area'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['prev_fn_area'], df['fn_area'])
    df['fn_area'] = np.where((df['fn_area'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['next_fn_area'], df['fn_area'])

    df['job_id'] = np.where((df['job_id'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['prev_job_id'], df['job_id'])
    df['job_id'] = np.where((df['job_id'].isnull()) & ((df['obj'].notnull())|(df['noc'].notnull())) & (df['employee_id'] == df['prev_employee_id']), df['next_job_id'], df['job_id'])

#run these loops again to maintain consistency in data.
seniority_fn()
fn_loop()

In [27]:
#Our last resort, if the employee has no previous or future information at all, is to copy the manager information.
df['bus_supp'] = np.where(df['bus_supp'].isnull(), df['manager_bus_supp'], df['bus_supp'])
df['fn_area'] = np.where(df['fn_area'].isnull(), df['manager_fn_area'], df['fn_area'])

Filling length service

In [28]:
mode_length_service = df[df['year'] == 2010]['length_service'].mode()[0]

def update3():
    df['prev_length_service'] = df['length_service'].shift()
    df['next_length_service'] = df['length_service'].shift(periods=-1)

In [29]:
#Use adjacent information (next year) to guess the correct value of length_service
for i in range(8):
    update3()
    df['length_service'] = np.where((df['length_service'].isnull()) & (df['employee_id'] == df['next_employee_id']) & (df['next_length_service'] > 0), df['next_length_service']-(df['next_year']-df['year']), df['length_service'])

#If the observation is in 2010 and is missing length_service, we fill it with the mode value,
    #since we can't know when the employee started in the company. If it's in another year, we know it'll likely be 0.
df['length_service'] = np.where((df['length_service'].isnull()) & (df['employee_id'] != df['prev_employee_id']) & (df['year'] == 2010), mode_length_service, df['length_service'])
df['length_service'] = np.where((df['length_service'].isnull()) & (df['employee_id'] != df['prev_employee_id']) & (df['year'] != 2010), 0, df['length_service'])

#Use adjacent information (previous year) to guess the correct value of length_service
for j in range(8):
    update3()
    df['length_service'] = np.where((df['length_service'].isnull()) & (df['prev_length_service'].notnull()) & (df['employee_id'] == df['prev_employee_id']), df['prev_length_service']+(df['year']-df['prev_year']), df['length_service'])

#Sometimes length_service this year is not consistent with previous year (it should be previous year + 1). 
    #Here we try to correct it.
for k in range(8):
    update3()
    df['length_service'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['prev_length_service'].notnull()) & (df['length_service'] != (df['prev_length_service']+(df['year']-df['prev_year']))), df['prev_length_service']+(df['year']-df['prev_year']), df['length_service'])

#### Filling job length and leftover bus_supp, fn_area and job_id

In [30]:
#We notice that the length_service data is reversed for some employees. Here we try to correct that.
for i in range(8):
    df['prev_seniority_job_mnths'] = df['seniority_job_mnths'].shift()
    df['next_seniority_job_mnths'] = df['seniority_job_mnths'].shift(periods=-1)
    
    df['seniority_job_mnths'] = np.where((df['employee_id'] == df['next_employee_id']) & (df['seniority_job_mnths'] == df['next_seniority_job_mnths']+(df['next_year']-df['year'])*12), df['next_seniority_job_mnths'], df['seniority_job_mnths'])

In [31]:
#We fill these columns with some placeholder unknown values, so we don't have to drop them when running our models.
df['bus_supp'] = np.where(df['bus_supp'].isnull(), 'Unknown', df['bus_supp'])
df['fn_area'] = np.where(df['fn_area'].isnull(), 'Not assigned', df['fn_area'])
#We are pretty confident that 'be56b1a06b8ca176c8da6514bfcb1054' means unknown job.
df['job_id'] = np.where(df['job_id'].isnull(), 'be56b1a06b8ca176c8da6514bfcb1054', df['job_id'])

In [32]:
#Here we try to recover more information. If the seniority_job_mnths are consistent between 2 adjacent records,
    #we are pretty confident that the job, bus_supp and fn_area are the same.
for j in range(8):
    update2()
    
    df['bus_supp'] = np.where((df['bus_supp'] == 'Unknown') & (df['employee_id'] == df['prev_employee_id']) & (df['seniority_job_mnths'] == df['prev_seniority_job_mnths']+(df['year']-df['prev_year'])*12), df['prev_bus_supp'], df['bus_supp'])
    df['bus_supp'] = np.where((df['bus_supp'] == 'Unknown') & (df['employee_id'] == df['next_employee_id']) & (df['seniority_job_mnths'] == df['next_seniority_job_mnths']-(df['next_year']-df['year'])*12), df['next_bus_supp'], df['bus_supp'])
    
    df['fn_area'] = np.where((df['fn_area'] == 'Not assigned') & (df['employee_id'] == df['prev_employee_id']) & (df['seniority_job_mnths'] == df['prev_seniority_job_mnths']+(df['year']-df['prev_year'])*12), df['prev_fn_area'], df['fn_area'])
    df['fn_area'] = np.where((df['fn_area'] == 'Not assigned') & (df['employee_id'] == df['next_employee_id']) & (df['seniority_job_mnths'] == df['next_seniority_job_mnths']-(df['next_year']-df['year'])*12), df['next_fn_area'], df['fn_area'])
    
    df['job_id'] = np.where((df['job_id'] == 'be56b1a06b8ca176c8da6514bfcb1054') & (df['employee_id'] == df['prev_employee_id']) & (df['seniority_job_mnths'] == df['prev_seniority_job_mnths']+(df['year']-df['prev_year'])*12), df['prev_job_id'], df['job_id'])
    df['job_id'] = np.where((df['job_id'] == 'be56b1a06b8ca176c8da6514bfcb1054') & (df['employee_id'] == df['next_employee_id']) & (df['seniority_job_mnths'] == df['next_seniority_job_mnths']-(df['next_year']-df['year'])*12), df['next_job_id'], df['job_id'])

In [33]:
#Filling the rest of seniority_job_mnths
for k in range(8):
    df['prev_seniority_job_mnths'] = df['seniority_job_mnths'].shift()
    df['next_seniority_job_mnths'] = df['seniority_job_mnths'].shift(periods=-1)
    
    df['seniority_job_mnths'] = np.where((df['seniority_job_mnths'].isnull()) & (df['employee_id'] == df['next_employee_id']) & (df['job_id'] == df['next_job_id']) & (df['next_seniority_job_mnths']>=(df['next_year']-df['year'])*12), df['next_seniority_job_mnths']-(df['next_year']-df['year'])*12, df['seniority_job_mnths'])

df['seniority_job_mnths'] = np.where((df['seniority_job_mnths'].isnull()) & ((df['job_id'] != df['prev_job_id'])|(df['employee_id'] != df['prev_employee_id'])), 0, df['seniority_job_mnths'])
    
for l in range(8):
    df['prev_seniority_job_mnths'] = df['seniority_job_mnths'].shift()
    df['next_seniority_job_mnths'] = df['seniority_job_mnths'].shift(periods=-1)
    
    df['seniority_job_mnths'] = np.where((df['seniority_job_mnths'].isnull()) & (df['employee_id'] == df['prev_employee_id']) & (df['job_id'] == df['prev_job_id']), df['prev_seniority_job_mnths']+(df['year']-df['prev_year'])*12, df['seniority_job_mnths'])
    df['seniority_job_mnths'] = np.where((df['seniority_job_mnths'].notnull()) & (df['employee_id'] == df['prev_employee_id']) & (df['job_id'] == df['prev_job_id']) & (df['seniority_job_mnths'] != df['prev_seniority_job_mnths']+(df['year']-df['prev_year'])*12), df['prev_seniority_job_mnths']+(df['year']-df['prev_year'])*12, df['seniority_job_mnths'])

### 3. Feature engineering: creating additional features from original data

In [34]:
df = df.loc[:,:'seniority_job_mnths']

In [35]:
#Create dummy columns for bus_supp
bus_supp_dummy = pd.get_dummies(df['bus_supp'])
bus_supp_dummy.columns = ['business', 'support', 'unknown']
df = pd.concat([df.loc[:,:'bus_supp'], bus_supp_dummy, df.loc[:,'fn_area':]], axis=1)

In [36]:
#Create an area code column based on the dictionary d at the beginning of the notebook.
df['fn_area_code'] = df['fn_area'].map(d)

In [37]:
#Create dummy columns for fn_area
fn_area_dummy = pd.get_dummies(df['fn_area_code'])
fn_area_dummy = fn_area_dummy.reindex(columns=sorted(fn_area_dummy.columns.values, key=lambda x: int(x[4:])))
df = pd.concat([df.loc[:,:'fn_area'], df['fn_area_code'], fn_area_dummy, df.loc[:,'job_id':'seniority_job_mnths']], axis=1)

In [38]:
#White collar here means the employee has a function class
df['white_collar'] = np.where(df['fn_class'].notnull(), 1, 0)

In [39]:
#fn_class 15-18: junior -- 19-22: middle -- 23-26:senior
df['fn_junior'] = np.where(df['fn_class'] < 19, 1, 0)
df['fn_middle'] = np.where((df['fn_class'] >= 19) & (df['fn_class'] <= 22), 1, 0)
df['fn_senior'] = np.where(df['fn_class'] > 22, 1, 0)

In [40]:
#The obj_noc variable is the average of obj and noc score, or one of them if the other is missing.
    #We don't use it in our models, but it is convenient to have a single performance score 
    #when we do further analysis later.
df['obj_noc'] = np.where(df['obj'].isnull() & df['noc'].notnull(), df['noc'], np.where(df['obj'].notnull() & df['noc'].isnull(), df['obj'], (df['obj']+df['noc'])/2))

#Create time series variables: score from year-1, year-2 and year-3. 
    #The next score is only for analysis after the modelling.
df['prev_obj'] = df['obj'].shift()
df['prev_obj2'] = df['obj'].shift(periods=2)
df['prev_obj3'] = df['obj'].shift(periods=3)
df['next_obj'] = df['obj'].shift(periods=-1)

df['prev_noc'] = df['noc'].shift()
df['prev_noc2'] = df['noc'].shift(periods=2)
df['prev_noc3'] = df['noc'].shift(periods=3)
df['next_noc'] = df['noc'].shift(periods=-1)

df['prev_employee_id'] = df['employee_id'].shift()
df['prev_manager_id'] = df['manager_id'].shift()
df['prev_fn_class'] = df['fn_class'].shift()
df['prev_fn_area'] = df['fn_area'].shift()
df['prev_bus_supp'] = df['bus_supp'].shift()
df['prev_job_id'] = df['job_id'].shift()

#Binary variables related to change
df['change_manager'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['manager_id'].notnull()) & (df['prev_manager_id'].notnull()) & (df['manager_id'] != df['prev_manager_id']), 1, 0)
df['change_fn_class'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['fn_class'].notnull()) & (df['prev_fn_class'].notnull()) & (df['fn_class'] > df['prev_fn_class']), 1, 0)
df['change_fn_area'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['fn_area'] != 'Not assigned') & (df['prev_fn_area'] != 'Not assigned') & (df['fn_area'] != df['prev_fn_area']), 1, 0)
df['change_bus_supp'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['bus_supp'] != 'Unknown') & (df['prev_bus_supp'] != 'Unknown') & (df['bus_supp'] != df['prev_bus_supp']), 1, 0)
df['change_job'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['job_id'] != 'be56b1a06b8ca176c8da6514bfcb1054') & (df['prev_job_id'] != 'be56b1a06b8ca176c8da6514bfcb1054') & (df['job_id'] != df['prev_job_id']), 1, 0)

#A neccesary step to clean up after using shifting column
for i in ['prev_obj', 'prev_noc', 'prev_manager_id', 'prev_job_id', 'prev_fn_class', 'prev_fn_area']:
    df[i] = np.where(df['employee_id'] != df['prev_employee_id'], np.nan, df[i])

#Shifting columns for year-2, year-3
for i in range(2,4):
    prev_id = 'prev_employee_id' + str(i)
    prev_obj = 'prev_obj' + str(i)
    prev_noc = 'prev_noc' + str(i)
    df[prev_id] = df['employee_id'].shift(periods=i)
    df[prev_obj] = np.where(df['employee_id'] != df[prev_id], np.nan, df[prev_obj])
    df[prev_noc] = np.where(df['employee_id'] != df[prev_id], np.nan, df[prev_noc])

In [41]:
df['next_employee_id'] = df['employee_id'].shift(periods=-1)
df['next_fn_class'] = df['fn_class'].shift(periods=-1)

#This is the variable we'll use as target to predict a promotion.
df['next_change_fn_class'] = df['change_fn_class'].shift(periods=-1)
df['next_change_fn_class'] = np.where(df['employee_id'] != df['next_employee_id'], np.nan, df['next_change_fn_class'])

df['fn_class_years'] = np.where(df['fn_class'].notnull(), 1, 0)

for i in range(8):
    df['prev_fn_class_years'] = df['fn_class_years'].shift()
    df['fn_class_years'] = np.where((df['employee_id'] == df['prev_employee_id']) & (df['fn_class'] == df['prev_fn_class']) & (df['fn_class_years'] == df['prev_fn_class_years']), df['fn_class_years']+1, df['fn_class_years'])

In [42]:
#Create more variables showing previous performance. For example: obj_bad_years is the number of years in a row 
    #the employee had a bad obj score.
import operator as op

for i in ['obj', 'noc']:
    for j in [('_bad_years', op.lt, op.ge), ('_avg_years', op.eq, op.ne), ('_good_years', op.gt, op.le)]:
        col_name = i + j[0]
        df[col_name] = np.where(df['employee_id'] == df['prev_employee_id'], 0, np.nan)

        df['temp_switch'] = 1
        for k in range(1,9):
            df['temp_prev'] = df[i].shift(periods=k)
            df['temp_prev_id'] = df['employee_id'].shift(periods=k)
    
            df[col_name] = np.where((df['employee_id'] == df['temp_prev_id']) & (df['temp_switch'] == 1) & (j[1](df['temp_prev'], 3)), df[col_name]+1, df[col_name])
            df['temp_switch'] = np.where((df['employee_id'] == df['temp_prev_id']) & (df['temp_switch'] == 1) & (j[2](df['temp_prev'], 3)), 0, df['temp_switch'])
        
        df[col_name] = np.where(df[i].isnull(), np.nan, df[col_name])
        df[col_name] = np.where((df[col_name].isnull()) & (df[i].notnull()), 0, df[col_name])

In [43]:
#Create a variable indicating whether the employee is a manager or not
list_managers = df.groupby(['manager_id', 'year']).size().reset_index(name='yes')
list_managers['yes'] = 1

mapper = list_managers.groupby(by=['manager_id', 'year'])['yes'].count().to_dict()
df['is_manager'] = df.set_index(['employee_id', 'year']).index.map(mapper.get)
df['is_manager'] = np.where(df['is_manager'] == 1, 1, 0)

man_emp = df[df['is_manager'] == 1]

#Create a variable showing the number of employees this employee is managing
mapper = df.groupby(by=['manager_id', 'year'])['employee_id'].count().to_dict()
df['num_emp_managing'] = df.set_index(['employee_id', 'year']).index.map(mapper.get)
df['num_emp_managing'] = np.where((df['num_emp_managing'].isnull()), 0, df['num_emp_managing'])

#The average obj and noc score of the employees this employee is managing
mapper = df.groupby(by=['manager_id', 'year'])['obj'].mean().to_dict()
df['obj_emp_managing'] = df.set_index(['employee_id', 'year']).index.map(mapper.get)

mapper = df.groupby(by=['manager_id', 'year'])['noc'].mean().to_dict()
df['noc_emp_managing'] = df.set_index(['employee_id', 'year']).index.map(mapper.get)

#The average scores from last years
mapper = df.groupby(by=['manager_id', 'year'])['prev_obj'].mean().to_dict()
df['prev_obj_emp_managing'] = df.set_index(['employee_id', 'year']).index.map(mapper.get)

mapper = df.groupby(by=['manager_id', 'year'])['prev_noc'].mean().to_dict()
df['prev_noc_emp_managing'] = df.set_index(['employee_id', 'year']).index.map(mapper.get)

#The fn_class of the manager of this employee
mapper = man_emp[['employee_id', 'year', 'fn_class']].set_index(['employee_id', 'year'])['fn_class'].to_dict()
df['manager_fn_class'] = df.set_index(['manager_id', 'year']).index.map(mapper.get)

#The last year performance of the manager of this employee
mapper = man_emp[['employee_id', 'year', 'prev_obj']].set_index(['employee_id', 'year'])['prev_obj'].to_dict()
df['manager_prev_obj'] = df.set_index(['manager_id', 'year']).index.map(mapper.get)

mapper = man_emp[['employee_id', 'year', 'prev_noc']].set_index(['employee_id', 'year'])['prev_noc'].to_dict()
df['manager_prev_noc'] = df.set_index(['manager_id', 'year']).index.map(mapper.get)

In [44]:
#Delete unneccesary columns
df = df.drop(['prev_employee_id', 'prev_employee_id2', 'prev_employee_id3', 'prev_manager_id', 'prev_fn_class', 'prev_fn_area', 'prev_bus_supp', 'prev_job_id', 'next_employee_id', 'next_fn_class', 'prev_fn_class_years', 'temp_switch', 'temp_prev', 'temp_prev_id'], axis=1)

#### Feature engineering: adding external economic data

In [45]:
#Adding more economic variables
econ_data = pd.read_csv('external_data.csv', delimiter=';', decimal=',')

In [46]:
econ_data

,year,gdp_growth_beg,gdp_growth_mid,stock_return_beg,stock_return_mid,roe,num_emp,avg_wage,medical_accident,lost_time_accident,emp_satisfaction,glassdoor
0,2010,0.7,2.2,0.245322,0.076294,11.10,28204,71479.22281,5.10,1.60,0.77,3.857
1,2011,2.6,1.9,0.249583,0.357248,26.70,16785,79773.60739,4.80,1.60,0.77,3.750
2,2012,1.7,1.9,0.156313,-0.159330,4.30,29121,48830.74070,2.90,0.80,0.77,3.750
3,2013,1.9,1.1,0.009012,0.078667,9.80,29103,79098.37474,2.59,0.81,0.72,3.750
4,2014,1.1,0.4,0.252949,0.093402,4.23,27146,78943.49075,1.06,0.80,0.72,3.667
5,2015,0.5,0.8,0.045056,0.004547,1.11,25909,76807.28704,0.97,0.98,0.75,3.833
6,2016,1.5,2.4,-0.075295,-0.235294,4.82,26350,77457.30550,0.77,0.75,0.77,3.703
7,2017,1.9,1.6,-0.155287,0.315495,6.39,27030,89974.10285,0.77,0.76,0.75,3.798


In [47]:
df = pd.merge(df, econ_data, how='left', on=['year'])

In [50]:
df.to_csv('dataset_imputed.csv', index=False)